In [ ]:
import numpy as np
import requests, gzip, os, hashlib
%pylab inline

#fetch data
path = '.'

def fetch(url):
    fp = os.path.join(path, hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp):
        with open(fp, "rb") as f:
            data = f.read()
    else:
        with open(fp, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(gzip.decompress(data), dtype=np.uint8).copy()

X = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28*28))
Y = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28*28))
Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

In [ ]:
# Train-Validation split

rand = np.arange(60000)
np.random.shuffle(rand)
train_no = rand[:50000]

val_no = np.setdiff1d(rand,train_no)

X_train, X_val = X[train_no, :], X[val_no, :]
Y_train, Y_val = Y[train_no], Y[val_no]

In [ ]:
# PCA

from sklearn.decomposition import PCA

# Write your implementation below


In [ ]:
# PCA followed by logistic regression

from sklearn.linear_model import LogisticRegression

# Write your implementation below


In [ ]:
# select number of components

num_components = np.arange(20, 201, 20)

# Write your implementation below
